In [ ]:
# 누락 파일 확인 + 진행률 표시

import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ──────────────────────────────
# 경로 설정
# ──────────────────────────────
BASE_DIR = Path(r"C:\Users\mungm\Documents\ai_engineer\genai_rag2\data")
EXCEL_FILE = BASE_DIR / "data_list.xlsx"
RAW_DIR = BASE_DIR / "raw"

# ──────────────────────────────
# 엑셀 파일 읽기
# ──────────────────────────────
df = pd.read_excel(EXCEL_FILE)

if "파일명" not in df.columns:
    raise ValueError("엑셀 파일에 '파일명' 컬럼이 존재하지 않습니다.")

file_names = df["파일명"].dropna().astype(str).tolist()

# ──────────────────────────────
# raw 폴더 내 실제 파일명 집합
# ──────────────────────────────
existing_files = {file.name for file in RAW_DIR.iterdir() if file.is_file()}

# ──────────────────────────────
# 진행률 표시하며 누락 파일 확인
# ──────────────────────────────
missing_files = []

for fname in tqdm(
    file_names,
    desc="파일 존재 여부 확인 중",
    total=len(file_names),
):
    if fname not in existing_files:
        missing_files.append(fname)

# ──────────────────────────────
# 결과 출력
# ──────────────────────────────
print("\n✅ raw 폴더에 존재하지 않는 파일 목록")
print("-" * 50)

for fname in missing_files:
    print(fname)

print(f"\n총 검사 파일 수: {len(file_names)}")
print(f"총 누락 파일 수: {len(missing_files)}")

파일 존재 여부 확인 중: 100%|██████████| 100/100 [00:00<?, ?it/s]


✅ raw 폴더에 존재하지 않는 파일 목록
--------------------------------------------------

총 검사 파일 수: 100
총 누락 파일 수: 0


In [ ]:
# df 는 이미 read_excel 등으로 로드되었다고 가정
# 예: df = pd.read_excel("data_list.xlsx")

# ──────────────────────────────
# 컬럼 매핑 정의
# ──────────────────────────────
column_mapping = {
    "공고 번호": "notice_id",
    "공고 차수": "notice_round",
    "사업명": "project_name",
    "사업 금액": "project_budget",
    "발주 기관": "ordering_agency",
    "공개 일자": "publish_date",
    "입찰 참여 시작일": "bid_start_date",
    "입찰 참여 마감일": "bid_end_date",
    "사업 요약": "project_summary",
    "파일형식": "file_type",
    "파일명": "file_name",
    "텍스트": "text",
}

# ──────────────────────────────
# 컬럼 존재 여부 검증
# ──────────────────────────────
missing_cols = set(column_mapping.keys()) - set(df.columns)
if missing_cols:
    raise ValueError(f"엑셀에 존재하지 않는 컬럼: {missing_cols}")

# ──────────────────────────────
# 컬럼 추출 + 이름 변경
# ──────────────────────────────
df_rfp = df[list(column_mapping.keys())].rename(columns=column_mapping)

# ──────────────────────────────
# 결과 확인
# ──────────────────────────────
print("✅ df_rfp 생성 완료")

✅ df_rfp 생성 완료


In [12]:
df_rfp.head(3)

,notice_id,notice_round,project_name,project_budget,ordering_agency,publish_date,bid_start_date,bid_end_date,project_summary,file_type,file_name,text
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaT,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\n \n \n \n제 안 요 청 서\n[ 2024년 대학 산학협력활동 실태조...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\n \nEIP3.0 고압가스 안전관리\n시스템 구축 용역\n2024....


In [16]:
df_rfp["project_budget"] = (
    df_rfp["project_budget"]
    .astype(float)  # 과학적 표기 → float
    .astype("Int64")  # 정수 (NaN 허용)
)
df_rfp.head(3)

,notice_id,notice_round,project_name,project_budget,ordering_agency,publish_date,bid_start_date,bid_end_date,project_summary,file_type,file_name,text,project_budget_kor
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000,한영대학,2024-10-04 13:51:23,NaT,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...,1억 3천만원
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\n \n \n \n제 안 요 청 서\n[ 2024년 대학 산학협력활동 실태조...,1억 2천만원
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\n \nEIP3.0 고압가스 안전관리\n시스템 구축 용역\n2024....,4천만원


In [ ]:
def format_budget_kor(amount: int) -> str:
    if pd.isna(amount):
        return None

    amount = int(amount)

    eok = amount // 100_000_000
    remainder = amount % 100_000_000
    cheon = remainder // 10_000_000  # 천만원 단위

    parts = []

    if eok > 0:
        parts.append(f"{eok}억")
    if cheon > 0:
        parts.append(f"{cheon}천만원")

    if not parts:
        return "0원"

    return " ".join(parts)


df_rfp["project_budget_kor"] = df_rfp["project_budget"].apply(format_budget_kor)
df_rfp.head(3)

,notice_id,notice_round,project_name,project_budget,ordering_agency,publish_date,bid_start_date,bid_end_date,project_summary,file_type,file_name,text,project_budget_kor
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000,한영대학,2024-10-04 13:51:23,NaT,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...,1억 3천만원
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\n \n \n \n제 안 요 청 서\n[ 2024년 대학 산학협력활동 실태조...,1억 2천만원
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\n \nEIP3.0 고압가스 안전관리\n시스템 구축 용역\n2024....,4천만원
